In [2]:
!unzip '/content/Dataset_perSampling_pernodeConfig-20250921T214557Z-1-001.zip'
!pip install chronos-forecasting pandas numpy scikit-learn torch

Archive:  /content/Dataset_perSampling_pernodeConfig-20250921T214557Z-1-001.zip
  inflating: Dataset_perSampling_pernodeConfig/temperature_data_60min_25.csv  
  inflating: Dataset_perSampling_pernodeConfig/temperature_data_5min_25.csv  
  inflating: Dataset_perSampling_pernodeConfig/temperature_data_15min_25.csv  
  inflating: Dataset_perSampling_pernodeConfig/temperature_data_30min_16.csv  
  inflating: Dataset_perSampling_pernodeConfig/temperature_data_5min_8.csv  
  inflating: Dataset_perSampling_pernodeConfig/temperature_data_45min_8.csv  
  inflating: Dataset_perSampling_pernodeConfig/temperature_data_30min_25.csv  
  inflating: Dataset_perSampling_pernodeConfig/temperature_data_60min_8.csv  
  inflating: Dataset_perSampling_pernodeConfig/temperature_data_45min_25.csv  
  inflating: Dataset_perSampling_pernodeConfig/temperature_data_45min_16.csv  
  inflating: Dataset_perSampling_pernodeConfig/temperature_data_60min_16.csv  
  inflating: Dataset_perSampling_pernodeConfig/temperatu

In [5]:
# pip install chronos-forecasting pandas numpy scikit-learn torch

import os
import math
import numpy as np
import pandas as pd
import torch
from chronos import BaseChronosPipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

# --------------------------------------------------------------------------------------
# CONFIG
# --------------------------------------------------------------------------------------
MODEL_ID = os.getenv("CHRONOS_MODEL", "amazon/chronos-bolt-small")
# Use "amazon/chronos-t5-small" for the original Chronos (sampling-based) instead of Bolt.
# Official model IDs: amazon/chronos-bolt-{tiny,mini,small,base}, amazon/chronos-t5-{tiny,mini,small,base}.
# (Chronos-Bolt is faster and direct multi-step; Chronos-T5 draws samples; both work here.)  # see README
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE = torch.bfloat16 if (DEVICE == "cuda") else torch.float32

DATA_FMT = "Dataset_perSampling_pernodeConfig/temperature_data_{samp}min_{num_nodes}.csv"
TRAIN_SLICE = ("2018-11-01", "2018-11-06")
TEST_SLICE  = ("2018-11-08", "2018-11-10")

NODE_LIST   = [8, 16, 25]
SAMP_LIST   = [5, 15, 30, 45, 60]

K_NEIGHBORS = 3         # top-k for ensemble similarity
ALPHA_BLEND = 0.60      # target vs. neighbors weight (60/40)
EPS = 1e-12

# --------------------------------------------------------------------------------------
# PIPELINE INIT + SINGLE-SERIES PRED
# --------------------------------------------------------------------------------------
def init_chronos_pipeline(model_id: str) -> BaseChronosPipeline:
    """
    Creates a Chronos/Chronos-Bolt pipeline. The README uses BaseChronosPipeline.from_pretrained(...)
    and predict_quantiles(context=..., prediction_length=..., quantile_levels=[...]).
    """
    pipe = BaseChronosPipeline.from_pretrained(
        model_id,
        device_map=DEVICE,            # "cuda" or "cpu"
        torch_dtype=DTYPE,            # bfloat16 on GPU, float32 on CPU
    )
    return pipe

def chronos_predict_mean(pipe: BaseChronosPipeline, series_1d: np.ndarray, p_steps: int) -> np.ndarray:
    """
    Deterministic point forecast via predict_quantiles (Chronos README).
    Returns the mean vector of shape [p_steps].
    """
    # Ensure 1D contiguous float tensor; handle NaNs via forward-fill/back-fill, then zeros (robustness)
    s = pd.Series(series_1d).ffill().bfill().fillna(0.0).to_numpy(dtype=np.float32)
    _, mean = pipe.predict_quantiles(
        context=torch.tensor(s),
        prediction_length=p_steps,
        quantile_levels=[0.5],  # we don't need the quantiles here; 'mean' is returned alongside
    )
    return mean[0].cpu().numpy()

# --------------------------------------------------------------------------------------
# ENSEMBLE SIMILARITY (model-agnostic blender; identical logic to your TimesFM version)
# --------------------------------------------------------------------------------------
def ensemble_similarity_forecast(train_df: pd.DataFrame,
                                 target_col: str,
                                 p_steps: int,
                                 forecaster_fn,
                                 k: int = K_NEIGHBORS,
                                 alpha: float = ALPHA_BLEND) -> np.ndarray:
    # Pearson |r| on training window
    corr = (
        train_df.select_dtypes(include=[np.number])
                .corr(method="pearson", min_periods=1)[target_col]
                .abs()
                .sort_values(ascending=False)
    )
    neighbors = [c for c in corr.index if c != target_col][:max(0, min(k, len(corr)-1))]
    w = corr[neighbors].to_numpy()
    w = w / (w.sum() + EPS)

    # Target forecast
    y_tgt = forecaster_fn(train_df[target_col].to_numpy(dtype=float), p_steps)
    y_ens = alpha * y_tgt

    # Neighbor forecasts + simple level alignment via mean ratio
    tgt_mean = float(train_df[target_col].mean())
    for node, wi in zip(neighbors, w):
        node_mean = float(train_df[node].mean())
        ratio = (tgt_mean / node_mean) if node_mean != 0 else 1.0
        y_n = forecaster_fn(train_df[node].to_numpy(dtype=float), p_steps) * ratio
        y_ens += (1.0 - alpha) * wi * y_n

    return y_ens

# --------------------------------------------------------------------------------------
# MAIN BENCHMARK LOOP
# --------------------------------------------------------------------------------------
def run_benchmark():
    print(f"Using model: {MODEL_ID} on {DEVICE} (dtype={DTYPE})")
    pipe = init_chronos_pipeline(MODEL_ID)

    approaches = [
        ("Baseline (No Covariates)", None),
        ("Ensemble Similarity", ensemble_similarity_forecast),
    ]

    rows = []  # metrics rows

    for approach_name, approach_fn in approaches:
        print(f"\n=== Approach: {approach_name} ===")
        for num_nodes in NODE_LIST:
            for samp in SAMP_LIST:
                try:
                    # Load & split
                    df = pd.read_csv(DATA_FMT.format(samp=samp, num_nodes=num_nodes))
                    df["ds"] = pd.to_datetime(df["ds"])
                    df = df.set_index("ds").sort_index()
                    train = df.loc[TRAIN_SLICE[0]:TRAIN_SLICE[1]]
                    test  = df.loc[TEST_SLICE[0]:TEST_SLICE[1]]

                    # 4-hour horizon regardless of sampling
                    p_steps = (4 * 60) // int(samp)
                    if p_steps <= 0:
                        raise ValueError(f"Invalid p_steps for samp={samp}")

                    y_true_mat, y_pred_mat = [], []

                    for target_col in train.columns:
                        # ground truth slice (guard if test shorter)
                        actual = test[target_col].to_numpy(dtype=float)[:p_steps]

                        if approach_fn is None:
                            # Baseline: Chronos per node, no covariates
                            pred = chronos_predict_mean(pipe, train[target_col].to_numpy(dtype=float), p_steps)
                        else:
                            # Ensemble similarity using the same forecaster
                            pred = approach_fn(
                                train_df=train,
                                target_col=target_col,
                                p_steps=p_steps,
                                forecaster_fn=lambda arr, L: chronos_predict_mean(pipe, arr, L),
                            )

                        # truncate/pad to p_steps for safety
                        pred = np.asarray(pred, dtype=float)
                        if len(pred) < p_steps:
                            pred = np.pad(pred, (0, p_steps - len(pred)))
                        else:
                            pred = pred[:p_steps]

                        y_true_mat.append(actual)
                        y_pred_mat.append(pred)

                    # Stack as [p_steps, n_nodes]
                    y_true = np.stack(y_true_mat, axis=1)[:p_steps, :]
                    y_pred = np.stack(y_pred_mat, axis=1)[:p_steps, :]

                    mae  = mean_absolute_error(y_true, y_pred)
                    rmse = math.sqrt(mean_squared_error(y_true, y_pred))
                    mape = mean_absolute_percentage_error(y_true, y_pred) * 100

                    rows.append({
                        "model": MODEL_ID,
                        "approach": approach_name,
                        "nodes": num_nodes,
                        "sampling": samp,
                        "MAE": mae,
                        "RMSE": rmse,
                        "MAPE": mape,
                    })

                    print(f"  nodes={num_nodes}, samp={samp} → MAE={mae:.5f}, RMSE={rmse:.5f}, MAPE={mape:.3f}%")

                except Exception as e:
                    print(f"  Error for nodes={num_nodes}, samp={samp}: {e}")

    # Summary tables
    df_metrics = pd.DataFrame(rows)
    print("\n" + "="*80)
    print("📊 MAE by approach")
    print("="*80)
    if not df_metrics.empty:
        print(
            df_metrics.pivot_table(values="MAE",
                                   index=["nodes","sampling"],
                                   columns="approach").round(5)
        )

        print("\n" + "="*80)
        print("📊 RMSE by approach")
        print("="*80)
        print(
            df_metrics.pivot_table(values="RMSE",
                                   index=["nodes","sampling"],
                                   columns="approach").round(5)
        )

        # Average improvement vs. baseline
        base = df_metrics[df_metrics["approach"]=="Baseline (No Covariates)"] \
                .set_index(["nodes","sampling"])
        enh  = df_metrics[df_metrics["approach"]=="Ensemble Similarity"] \
                .set_index(["nodes","sampling"])
        common_idx = base.index.intersection(enh.index)
        if len(common_idx) > 0:
            mae_impr  = ((base.loc[common_idx,"MAE"] - enh.loc[common_idx,"MAE"]) / base.loc[common_idx,"MAE"] * 100).mean()
            rmse_impr = ((base.loc[common_idx,"RMSE"] - enh.loc[common_idx,"RMSE"]) / base.loc[common_idx,"RMSE"] * 100).mean()
            print("\n" + "="*80)
            print("📈 Avg improvement vs. Baseline")
            print("="*80)
            print(f"  MAE:  {mae_impr:+.2f}%")
            print(f"  RMSE: {rmse_impr:+.2f}%")

    # Optional: save
    out_csv = f"chronos_results_{MODEL_ID.split('/')[-1]}.csv"
    df_metrics.to_csv(out_csv, index=False)
    print(f"\n✅ Saved metrics to: {os.path.abspath(out_csv)}")

if __name__ == "__main__":
    run_benchmark()

Using model: amazon/chronos-bolt-small on cpu (dtype=torch.float32)

=== Approach: Baseline (No Covariates) ===
  nodes=8, samp=5 → MAE=3.84171, RMSE=4.42047, MAPE=52.701%
  nodes=8, samp=15 → MAE=3.69946, RMSE=4.20794, MAPE=51.003%
  nodes=8, samp=30 → MAE=3.50675, RMSE=4.12125, MAPE=50.265%
  nodes=8, samp=45 → MAE=3.15881, RMSE=3.78143, MAPE=40.189%
  nodes=8, samp=60 → MAE=4.04190, RMSE=4.50528, MAPE=51.268%
  nodes=16, samp=5 → MAE=3.73091, RMSE=4.33119, MAPE=50.920%
  nodes=16, samp=15 → MAE=3.80998, RMSE=4.37586, MAPE=52.119%
  nodes=16, samp=30 → MAE=3.56554, RMSE=4.12935, MAPE=49.982%
  nodes=16, samp=45 → MAE=3.17334, RMSE=3.76189, MAPE=40.291%
  nodes=16, samp=60 → MAE=4.15440, RMSE=4.58101, MAPE=52.116%
  nodes=25, samp=5 → MAE=3.43723, RMSE=4.00363, MAPE=46.732%
  nodes=25, samp=15 → MAE=3.44602, RMSE=3.99566, MAPE=46.911%
  nodes=25, samp=30 → MAE=3.32190, RMSE=3.86534, MAPE=46.321%
  nodes=25, samp=45 → MAE=2.97497, RMSE=3.58149, MAPE=37.662%
  nodes=25, samp=60 → MAE=3.